In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.4 MB/s eta 0:00:00


In [2]:
import bitsandbytes
import torch
import transformers
import peft

In [3]:
model_id = 'google-bert/bert-base-uncased'

dataset_id = 'Sp1786/multiclass-sentiment-analysis-dataset'

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit= True,
                                bnb_4bit_compute_dtype=torch.float16,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True)

model = AutoModelForSequenceClassification.from_pretrained(model_id, quantization_config = bnb_config, device_map='auto', num_labels=3)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(r = 8,
                         lora_alpha = 16,
                         lora_dropout= 0.05,
                         target_modules = ['query','value','key'],
                         bias = 'none',
                         task_type = TaskType.SEQ_CLS

)

model = get_peft_model(model, lora_config)

In [9]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)

dataset = dataset.filter(lambda row: row['text'] is not None)

def tokenization(example):
    text = example['text']
    return tokenizer(text, truncation=True)

dataset = dataset.map(tokenization, batched=True)

README.md: 0.00B [00:00, ?B/s]

train_df.csv: 0.00B [00:00, ?B/s]

val_df.csv: 0.00B [00:00, ?B/s]

test_df.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/5205 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5206 [00:00<?, ? examples/s]

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [ ]:
max = -10000
for i in range(len(dataset['train'])):
    if len(dataset['train'][i]['text'])>max:
        max=len(dataset['train'][i]['text'])

print(max)

2176


In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [11]:
from transformers import Trainer, DataCollatorWithPadding, TrainingArguments

train_args = TrainingArguments(output_dir='./bert-sentiment_analyzer',
                               overwrite_output_dir=True,
                               eval_strategy='epoch',
                               per_device_train_batch_size=64,
                               per_device_eval_batch_size=64,
                               gradient_accumulation_steps=2,
                               learning_rate=0.00005,
                               weight_decay=0.0001,
                               num_train_epochs=8,
                               save_strategy='epoch',
                               metric_for_best_model='accuracy',
                               load_best_model_at_end=True,
                               logging_steps=10)

trainer = Trainer(
    model=model,
    args=train_args,
    eval_dataset=dataset['validation'],
    train_dataset=dataset['train'],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdullahrashid (abdullahrashid-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.812000,0.791016,0.644957,0.642439,0.644957,0.641141
2,0.690400,0.687988,0.697406,0.699718,0.697406,0.696468
3,0.674100,0.664551,0.709510,0.709389,0.709510,0.708350
4,0.617200,0.653809,0.716619,0.718039,0.716619,0.715431
5,0.605000,0.644043,0.722767,0.723630,0.722767,0.722116
6,0.622600,0.643066,0.721806,0.721191,0.721806,0.719870
7,0.619900,0.640137,0.727762,0.729905,0.727762,0.727280
8,0.679200,0.637695,0.726417,0.726464,0.726417,0.725527


TrainOutput(global_step=1952, training_loss=0.6965215714251409, metrics={'train_runtime': 1313.6781, 'train_samples_per_second': 190.196, 'train_steps_per_second': 1.486, 'total_flos': 1.5846022326177792e+16, 'train_loss': 0.6965215714251409, 'epoch': 8.0})

In [40]:
trainer.evaluate(eval_dataset = dataset['test'])

{'eval_loss': 0.6298828125,
 'eval_accuracy': 0.7337175792507205,
 'eval_precision': 0.7330734880099564,
 'eval_recall': 0.7337175792507205,
 'eval_f1': 0.732006273627813,
 'eval_runtime': 13.6517,
 'eval_samples_per_second': 381.271,
 'eval_steps_per_second': 6.007,
 'epoch': 8.0}

In [38]:
text = "i love transformers"

inputs = tokenizer(text, return_tensors="pt", truncation=True).to(model.device)

with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=-1)

id2label = {0: "negative", 1: "neutral", 2: "positive"}
print("Prediction:", id2label[preds.item()])

Prediction: positive
